In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [2]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
house_prices_df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [3]:
house_prices_df = pd.concat([house_prices_df,pd.get_dummies(house_prices_df.mszoning, prefix="mszoning", drop_first=True)], axis=1)
house_prices_df = pd.concat([house_prices_df,pd.get_dummies(house_prices_df.street, prefix="street", drop_first=True)], axis=1)
dummy_column_names = list(pd.get_dummies(house_prices_df.mszoning, prefix="mszoning", drop_first=True).columns)
dummy_column_names = dummy_column_names + list(pd.get_dummies(house_prices_df.street, prefix="street", drop_first=True).columns)

In [4]:
house_prices_df['totalsf'] = house_prices_df['totalbsmtsf'] + house_prices_df['firstflrsf'] + house_prices_df['secondflrsf']

house_prices_df['int_over_sf'] = house_prices_df['totalsf'] * house_prices_df['overallqual']

# Y is the target variable
Y = np.log1p(house_prices_df['saleprice'])
# X is the feature set
X = house_prices_df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalsf', 'int_over_sf'] + dummy_column_names]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

alphas = [np.power(10.0,p) for p in np.arange(-10,40,1)]

In [5]:
lrm = LinearRegression()

lrm.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

print("R-squared of the model in training set is: {}".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.8321322553132751
-----Test set statistics-----
R-squared of the model in test set is: 0.8249302330916463
Mean absolute error of the prediction is: 0.12570372872861446
Mean squared error of the prediction is: 0.029192121871356457
Root mean squared error of the prediction is: 0.1708570217209596
Mean absolute percentage error of the prediction is: 1.0503577667823543


In [6]:
lasso_cv = LassoCV(alphas=alphas, cv=5)

lasso_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lasso_cv.predict(X_train)
y_preds_test = lasso_cv.predict(X_test)

print("Best alpha value is: {}".format(lasso_cv.alpha_))
print("R-squared of the model in training set is: {}".format(lasso_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lasso_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 0.0001
R-squared of the model in training set is: 0.831939428704242
-----Test set statistics-----
R-squared of the model in test set is: 0.8226434437869412
Mean absolute error of the prediction is: 0.12624310826908416
Mean squared error of the prediction is: 0.029573434037677038
Root mean squared error of the prediction is: 0.17196928225028166
Mean absolute percentage error of the prediction is: 1.0552354946577736


In [7]:
ridge_cv = RidgeCV(alphas=alphas, cv=5)

ridge_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridge_cv.predict(X_train)
y_preds_test = ridge_cv.predict(X_test)

print("Best alpha value is: {}".format(ridge_cv.alpha_))
print("R-squared of the model in training set is: {}".format(ridge_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(ridge_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 1.0
R-squared of the model in training set is: 0.8316364867222638
-----Test set statistics-----
R-squared of the model in test set is: 0.8203050076234278
Mean absolute error of the prediction is: 0.12673637339741076
Mean squared error of the prediction is: 0.02996335809297898
Root mean squared error of the prediction is: 0.17309927236409453
Mean absolute percentage error of the prediction is: 1.0596941230310677


In [8]:
elasticnet_cv = ElasticNetCV(alphas=alphas, cv=5)

elasticnet_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = elasticnet_cv.predict(X_train)
y_preds_test = elasticnet_cv.predict(X_test)

print("Best alpha value is: {}".format(elasticnet_cv.alpha_))
print("R-squared of the model in training set is: {}".format(elasticnet_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(elasticnet_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 0.001
R-squared of the model in training set is: 0.8299654806803803
-----Test set statistics-----
R-squared of the model in test set is: 0.8149185869526183
Mean absolute error of the prediction is: 0.12770726087011366
Mean squared error of the prediction is: 0.03086152030253385
Root mean squared error of the prediction is: 0.17567447254092966
Mean absolute percentage error of the prediction is: 1.0685444897303116


In [9]:
scores = cross_val_score(lrm, X_test, y_test, cv=10)
print("Explained Variance Scores: {}.".format(scores))
print("Coefficient of Determination: {:.2f}% (+/- {:.2f}%).".format(scores.mean()*100, scores.std() * 200))

Explained Variance Scores: [0.89499393 0.873937   0.85673218 0.80062964 0.82497089 0.5790531
 0.83555855 0.85573187 0.75871782 0.87472304].
Coefficient of Determination: 81.55% (+/- 17.48%).


In [10]:
scores = cross_val_score(lasso_cv, X_test, y_test, cv=10)
print("Explained Variance Scores: {}.".format(scores))
print("Coefficient of Determination: {:.2f}% (+/- {:.2f}%).".format(scores.mean()*100, scores.std() * 200))

Explained Variance Scores: [0.89488579 0.87364229 0.85673218 0.8012185  0.82468176 0.57827846
 0.83544511 0.85629645 0.76050961 0.83286857].
Coefficient of Determination: 81.15% (+/- 17.11%).


In [11]:
scores = cross_val_score(ridge_cv, X_test, y_test, cv=10)
print("Explained Variance Scores: {}.".format(scores))
print("Coefficient of Determination: {:.2f}% (+/- {:.2f}%).".format(scores.mean()*100, scores.std() * 200))

Explained Variance Scores: [0.89499791 0.87385683 0.85673236 0.80086894 0.82494539 0.57880347
 0.83546522 0.85592231 0.75920138 0.86184081].
Coefficient of Determination: 81.43% (+/- 17.33%).


According to the results, the best model is the OLS regression.